# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 2:39PM,238099,10,MET210595,"Methapharm, Inc.",Released
1,May 25 2022 2:39PM,238099,10,MET210596,"Methapharm, Inc.",Released
2,May 25 2022 2:39PM,238099,10,MET210597,"Methapharm, Inc.",Released
3,May 25 2022 2:39PM,238099,10,MET210598,"Methapharm, Inc.",Released
4,May 25 2022 2:31PM,238098,21,26684549,"NBTY Global, Inc.",Released
5,May 25 2022 2:31PM,238094,20,ALUR460040410,Alumier Labs Inc.,Released
6,May 25 2022 2:31PM,238094,20,ALU0006915,Alumier Labs Inc.,Released
7,May 25 2022 2:31PM,238094,20,ALU0006916,Alumier Labs Inc.,Released
8,May 25 2022 2:31PM,238094,20,ALU0006917,Alumier Labs Inc.,Released
9,May 25 2022 2:31PM,238094,20,ALUR944583403,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238095,Released,28
31,238096,Released,4
32,238097,Released,1
33,238098,Released,1
34,238099,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238095,NaN,NaN,28.0
238096,NaN,NaN,4.0
238097,NaN,NaN,1.0
238098,NaN,NaN,1.0
238099,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0.0,26.0,9.0
238033,0.0,0.0,2.0
238037,19.0,1.0,0.0
238042,0.0,1.0,0.0
238045,6.0,16.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0,26,9
238033,0,0,2
238037,19,1,0
238042,0,1,0
238045,6,16,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,0,26,9
1,238033,0,0,2
2,238037,19,1,0
3,238042,0,1,0
4,238045,6,16,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,,26,9
1,238033,,,2
2,238037,19,1,
3,238042,,1,
4,238045,6,16,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 2:39PM,238099,10,"Methapharm, Inc."
4,May 25 2022 2:31PM,238098,21,"NBTY Global, Inc."
5,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.
45,May 25 2022 2:25PM,238095,10,ISDIN Corporation
73,May 25 2022 2:25PM,238097,10,ISDIN Corporation
74,May 25 2022 2:25PM,238096,18,Hush Hush
78,May 25 2022 2:03PM,238093,21,"NBTY Global, Inc."
79,May 25 2022 2:01PM,238092,22,"NBTY Global, Inc."
80,May 25 2022 1:53PM,238091,10,"Nextsource Biotechnology, LLC"
82,May 25 2022 1:49PM,238090,20,Reliable 1 Laboratories LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",,,4
1,May 25 2022 2:31PM,238098,21,"NBTY Global, Inc.",,,1
2,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,,,40
3,May 25 2022 2:25PM,238095,10,ISDIN Corporation,,,28
4,May 25 2022 2:25PM,238097,10,ISDIN Corporation,,,1
5,May 25 2022 2:25PM,238096,18,Hush Hush,,,4
6,May 25 2022 2:03PM,238093,21,"NBTY Global, Inc.",,,1
7,May 25 2022 2:01PM,238092,22,"NBTY Global, Inc.",,,1
8,May 25 2022 1:53PM,238091,10,"Nextsource Biotechnology, LLC",,,2
9,May 25 2022 1:49PM,238090,20,Reliable 1 Laboratories LLC,,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4,,
1,May 25 2022 2:31PM,238098,21,"NBTY Global, Inc.",1,,
2,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,40,,
3,May 25 2022 2:25PM,238095,10,ISDIN Corporation,28,,
4,May 25 2022 2:25PM,238097,10,ISDIN Corporation,1,,
5,May 25 2022 2:25PM,238096,18,Hush Hush,4,,
6,May 25 2022 2:03PM,238093,21,"NBTY Global, Inc.",1,,
7,May 25 2022 2:01PM,238092,22,"NBTY Global, Inc.",1,,
8,May 25 2022 1:53PM,238091,10,"Nextsource Biotechnology, LLC",2,,
9,May 25 2022 1:49PM,238090,20,Reliable 1 Laboratories LLC,29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4,,
1,May 25 2022 2:31PM,238098,21,"NBTY Global, Inc.",1,,
2,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,40,,
3,May 25 2022 2:25PM,238095,10,ISDIN Corporation,28,,
4,May 25 2022 2:25PM,238097,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4.0,NaN,NaN
1,May 25 2022 2:31PM,238098,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,40.0,NaN,NaN
3,May 25 2022 2:25PM,238095,10,ISDIN Corporation,28.0,NaN,NaN
4,May 25 2022 2:25PM,238097,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 2:39PM,238099,10,"Methapharm, Inc.",4.0,0.0,0.0
1,May 25 2022 2:31PM,238098,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,May 25 2022 2:31PM,238094,20,Alumier Labs Inc.,40.0,0.0,0.0
3,May 25 2022 2:25PM,238095,10,ISDIN Corporation,28.0,0.0,0.0
4,May 25 2022 2:25PM,238097,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3333050,70.0,23.0,25.0
16,238079,1.0,0.0,0.0
18,476129,6.0,0.0,0.0
19,714197,1.0,2.0,0.0
20,952283,85.0,26.0,0.0
21,476191,2.0,0.0,0.0
22,714222,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3333050,70.0,23.0,25.0
1,16,238079,1.0,0.0,0.0
2,18,476129,6.0,0.0,0.0
3,19,714197,1.0,2.0,0.0
4,20,952283,85.0,26.0,0.0
5,21,476191,2.0,0.0,0.0
6,22,714222,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,70.0,23.0,25.0
1,16,1.0,0.0,0.0
2,18,6.0,0.0,0.0
3,19,1.0,2.0,0.0
4,20,85.0,26.0,0.0
5,21,2.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,70.0
1,16,Released,1.0
2,18,Released,6.0
3,19,Released,1.0
4,20,Released,85.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21,22
Status,,,,,,,
Completed,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,23.0,0.0,0.0,2.0,26.0,0.0,0.0
Released,70.0,1.0,6.0,1.0,85.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21,22
0,Completed,25.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,0.0,2.0,26.0,0.0,0.0
2,Released,70.0,1.0,6.0,1.0,85.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21,22
0,Completed,25.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,0.0,2.0,26.0,0.0,0.0
2,Released,70.0,1.0,6.0,1.0,85.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()